In [23]:
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.0+cu117


'%.4f'

In [24]:
lib.utils.set_seed()
train_x, train_y = lib.datasets.get_linear2()
network = lib.nets.create_livenet_linear2(l1=0.1)
# network = lib.nets.LINEAR3(l1=0.1)

In [25]:
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=len(train_x))
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=3000, adaptive_lr=False)


LiveNet


In [29]:

trainer.step(2000)


Iˈ0.000 0.571 = 0.224+0.348 lib/trainer.py:89


In [27]:
pred = network(train_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = train_y - pred_bin
len(diff[diff != 0]) / len(diff)

0.0000

In [6]:
importlib.reload(lib)
%matplotlib
plt.ion()

def param_picker0(param):
    val0 = param["params"]["linear1.weight"][0][0].item()
    val1 = param["params"]["linear1.weight"][1][0].item()
    return val1 + val0

def param_picker1(param):
    val0 = param["params"]["1->2"].item()
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_picker1)
plt.plot(values)
plt.grid()

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
# accum.plot()

Using matplotlib backend: TkAgg


KeyError: '1->2'

In [30]:
for name, p in network.named_parameters():
    print(name, f"{p.item():.6f}")

D0 0.927486
D1 -0.946418
S1->D1 3.772856


In [21]:
param = nn.Parameter(torch.tensor(0.0))
# network.register_parameter("S0->D0", param)
network._parameters.pop("S0->D0")
# network._parameters


KeyError: 'S0->D0'

S1->D1 0.927462
S1->D1 -0.946442
S1->D1 -0.750392
S1->D1 -0.771363
S1->D1 3.772857
